In [ ]:
#pymysql : mariadb 접속 프로그램(패키지)
#loopback ip 로컬에서쓰는 ip = host='127.0.0.1'
#autocommit=True -> insert into, update set, delete from한뒤 commit수작업을 안해줘도 된다.

In [ ]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', 
                       password='10042356', use_unicode=True, db="sungjuk_test", charset='utf8', autocommit=True)
cursor = conn.cursor()
cursor.execute("""
Create table persons(
id INT NOT NULL,
name VARCHAR(100),
salesrep VARCHAR(100),
PRIMARY KEY(id)
)
""")

In [ ]:
cur.executemany( #여러개일때는 many 하나일때는.execute
    "INSERT into persons values( %s, %s, %s)",
    [(1,"부산만","매니저"),
    (2, "트럼프","문재인"),
    (3, "아베", "문재인")])
conn.commit()

In [12]:
cursor.execute('select * from persons where name = %s', '아베') #포맷(쿼리문)을 유지하면서 찾고자하는 값만 바꾸고 싶은 경우를 위해
#cursor를 실행하면 recordset이 넘어오고
row = cursor.fetchone() #한개의 데이터를 가져온다.
print("ID=%d, Name=%s"% (row[0],row[1])) #인덱스로 데이터를 접근하고 있다.
print(row)

ID=3, Name=아베
(3, '아베', '문재인')


In [ ]:
cursor.execute("""
Create table pet(
id INT NOT NULL AUTO_INCREMENT,
name VARCHAR(50) NOT NULL,
owner VARCHAR(50),
species VARCHAR(50),
sex CHAR(2),
birth DATE,
death DATE,
PRIMARY KEY(id)
)
""")

In [ ]:
cursor.executemany( 
    "INSERT into pet(name, owner, species, sex, birth, death) values( %s, %s, %s, %s, %s, %s)",
    [("인천시","민주", "cat", "f", '2001-02-04', None),
    ("대구시", "자유", "cat", "f", '2010-03-17', None),
    ("대전시", "민주", "dog", "f", '2010-05-13', None),
    ("광주시", "민주", "dog", "m", '2015-08-27', None),
    ("부산시", "자유", "dog", "m", '2017-08-31', '2018-04-29')])
conn.commit()

In [ ]:
#birth 가 2015.1.1 일 이후인 데이터를 출력하시오
#dog이면서 'f'인 데이터를 출력하시오
#name과 birth만 birth순으로 내림차순 정렬하여 출력하시오
#death가 null인 데이터만 출력하시오
cursor.execute("""
select * from pet where birth>='2015-01-01';
""")
row = cursor.fetchall()
pet_select = list(row) #받아온 데이터를 list에 저장해서 출력해야 보기가 편하다.
for per in pet_select:
    print(per)

In [ ]:
cursor.execute("""
select * from pet where species='dog' and sex='f';
""")
row2 = cursor.fetchall()
pet_select = list(row)
for per in pet_select:
    print(per)

In [ ]:
cursor.execute("""
select name,birth from pet order by birth desc;
""")
row2 = cursor.fetchall()
pet_select = list(row)
for per in pet_select:
    print(per)

In [ ]:
cursor.execute("""
select * from pet where death is null;
""")
row2 = cursor.fetchall()
pet_select = list(row)
for per in pet_select:
    print(per)
conn.commit()

In [ ]:
conn = pymysql.connect(host='127.0.0.1', user='root', port=3306, password='10042356', db='sungjuk_test', charset='utf8', 
                      cursorclass=pymysql.cursors.DictCursor, autocommit=True)
cur = conn.cursor()

In [ ]:
cur.execute('select * from pet')
print(cur.rowcount) #쿼리의 상태를 알수있다.(rowcount) 성공여부를 알수있다.
cur.description

In [ ]:
cur.fetchone() #하나씩 갖고옴
cur.fetchone()
cur.fetchone()
recs=cur.fetchmany(2) #다지나가면 안갖고옴 다시갖고 와야한다. 한번 넘어가면 없어진다.(순환하지 않는다.)
for rec in recs:
    print(rec)

In [ ]:
resc = cur.fetchall()
for rec in recs:
    print(rec)
cur.close() #반드시 커서와 connection 객체는 close해야한다. : os자원을 낭비하기 때문에
conn.close() #close한 다음에는 다시 생성해야한다.

In [1]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='root', port=3306, password='10042356', db='sungjuk_test', charset='utf8', 
                      cursorclass=pymysql.cursors.DictCursor, autocommit=True)
cur = conn.cursor()

In [2]:
cur.callproc('pet_all') #디폴트가 리스트 DictCursor 딕트형식으로 뽑혀져 온다.
for row in cur:
    print("ID=%s, name=%s" %(row["id"], row["name"]))

ID=1, name=인천시
ID=2, name=대구시
ID=3, name=대전시
ID=4, name=광주시
ID=5, name=부산시
ID=6, name=인천시
ID=7, name=부산시
ID=8, name=부산시


In [3]:
#문제 insert procedue을 만들고 다음data를 입력하시오.
data = ("인천시", "자유", "dog", "m", "2017-08-31", "2018-04-29", 0) #result의 공간을 채워서 보내야 한다.
data2 = ("부산시", "민주", "cat", "m", "2001-02-04", "2019-12-31", 0)
cur.callproc('pet_insert', data2)
cur.callproc('pet_all')
for row in cur:
    print("ID=%s, name=%s" %(row["id"], row["name"]))

ID=1, name=인천시
ID=2, name=대구시
ID=3, name=대전시
ID=4, name=광주시
ID=5, name=부산시
ID=6, name=인천시
ID=7, name=부산시
ID=8, name=부산시
ID=9, name=부산시


In [22]:
#문제
#update 프로시져를 만들고 부산시의 이름을 부산항으로 바꾸시오.
#만약 입력이 성공하면 성공했습니다. 라는 메세지를 출력하시오.
data = ("인천시","부산항",1)
cur.callproc('pet_updatename', data)
cur.execute('select @_pet_updatename_2')
result = cur.fetchone()
if result['@_pet_updatename_2']==-1:
    print("실행중 에러가 발생했습니다.")
elif result['@_pet_updatename_2']==0:
    print("성공하였습니다.")
else: print("실패했습니다.")
for row in cur:
    print(row)

실패했습니다.
